# Image Denoising using Wavelet Transform
## Lab 5: Wavelet-based Image Denoising

This notebook demonstrates:
1. Adding noise to images at various SNR levels (0 dB, 10 dB, 20 dB)
2. Denoising using hard and soft thresholding in wavelet domain
3. Parameter analysis (wavelet type, decomposition levels, thresholding type)
4. Translation Invariant Wavelet Transform (TI-WT) for improved denoising

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import pywt
from skimage import io, color, img_as_float
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from skimage.util import random_noise
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
%matplotlib inline

## 1. Utility Functions

In [ ]:
def add_noise_snr(image, snr_db):
    """
    Add Gaussian noise to image at specified SNR level
    
    Parameters:
    -----------
    image : ndarray
        Clean input image
    snr_db : float
        Signal-to-noise ratio in dB
    
    Returns:
    --------
    noisy_image : ndarray
        Image with added Gaussian noise
    """
    # Calculate signal power
    signal_power = np.mean(image ** 2)
    
    # Calculate noise power from SNR
    snr_linear = 10 ** (snr_db / 10)
    noise_power = signal_power / snr_linear
    
    # Generate Gaussian noise
    noise = np.random.normal(0, np.sqrt(noise_power), image.shape)
    
    # Add noise to image
    noisy_image = image + noise
    
    # Clip values to valid range [0, 1]
    noisy_image = np.clip(noisy_image, 0, 1)
    
    return noisy_image

def calculate_snr(original, noisy):
    """
    Calculate actual SNR between original and noisy image
    """
    signal_power = np.mean(original ** 2)
    noise_power = np.mean((original - noisy) ** 2)
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

def hard_threshold(coeffs, threshold):
    """
    Apply hard thresholding to wavelet coefficients
    
    Hard thresholding: Set coefficients below threshold to zero
    """
    return pywt.threshold(coeffs, threshold, mode='hard')

def soft_threshold(coeffs, threshold):
    """
    Apply soft thresholding to wavelet coefficients
    
    Soft thresholding: Shrink coefficients towards zero
    """
    return pywt.threshold(coeffs, threshold, mode='soft')

def estimate_noise_sigma(noisy_image, wavelet='db4'):
    """
    Estimate noise standard deviation using MAD (Median Absolute Deviation)
    in the finest wavelet scale (HH subband)
    """
    coeffs = pywt.dwt2(noisy_image, wavelet)
    HH = coeffs[1][2]  # Diagonal detail coefficients
    sigma = np.median(np.abs(HH)) / 0.6745
    return sigma

## 2. Load and Prepare Image

In [ ]:
# Load image - you can replace this with your own image path
# For demonstration, we'll use a sample image or create one

try:
    # Try to load a local image (replace with your image path)
    image = io.imread('your_image.png')
    
    # Convert to grayscale if needed
    if len(image.shape) == 3:
        image = color.rgb2gray(image)
    
    # Convert to float [0, 1]
    image = img_as_float(image)
    
except:
    # If no image available, use a sample image from skimage
    from skimage import data
    image = img_as_float(data.camera())  # Classic test image
    print("Using sample camera image")

# Display original image
plt.figure(figsize=(8, 8))
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.colorbar()
plt.axis('off')
plt.show()

print(f"Image shape: {image.shape}")
print(f"Image range: [{image.min():.3f}, {image.max():.3f}]")

## 3. Generate Noisy Images at Different SNR Levels

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Define SNR levels
snr_levels = [0, 10, 20]  # in dB

# Generate noisy images
noisy_images = {}
for snr in snr_levels:
    noisy_images[snr] = add_noise_snr(image, snr)

# Display noisy images
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

axes[0].imshow(image, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

for i, snr in enumerate(snr_levels):
    axes[i+1].imshow(noisy_images[snr], cmap='gray')
    psnr = peak_signal_noise_ratio(image, noisy_images[snr])
    actual_snr = calculate_snr(image, noisy_images[snr])
    axes[i+1].set_title(f'SNR = {snr} dB\nPSNR = {psnr:.2f} dB\nActual SNR = {actual_snr:.2f} dB')
    axes[i+1].axis('off')

plt.tight_layout()
plt.show()

## 4. Wavelet Denoising with Hard and Soft Thresholding

In [ ]:
def wavelet_denoise(noisy_image, wavelet='db4', level=3, threshold_mode='soft'):
    """
    Denoise image using wavelet transform with thresholding
    
    Parameters:
    -----------
    noisy_image : ndarray
        Noisy input image
    wavelet : str
        Wavelet type (e.g., 'db4', 'sym8', 'haar')
    level : int
        Number of decomposition levels
    threshold_mode : str
        'soft' or 'hard' thresholding
    
    Returns:
    --------
    denoised : ndarray
        Denoised image
    """
    # Estimate noise level
    sigma = estimate_noise_sigma(noisy_image, wavelet)
    
    # Perform wavelet decomposition
    coeffs = pywt.wavedec2(noisy_image, wavelet, level=level)
    
    # Calculate universal threshold
    # Universal threshold = sigma * sqrt(2 * log(N))
    N = noisy_image.size
    threshold = sigma * np.sqrt(2 * np.log(N))
    
    # Apply thresholding to detail coefficients (not approximation)
    coeffs_thresh = [coeffs[0]]  # Keep approximation coefficients
    
    for i in range(1, len(coeffs)):
        # coeffs[i] is a tuple of (cH, cV, cD)
        cH, cV, cD = coeffs[i]
        
        if threshold_mode == 'soft':
            cH_thresh = soft_threshold(cH, threshold)
            cV_thresh = soft_threshold(cV, threshold)
            cD_thresh = soft_threshold(cD, threshold)
        else:  # hard
            cH_thresh = hard_threshold(cH, threshold)
            cV_thresh = hard_threshold(cV, threshold)
            cD_thresh = hard_threshold(cD, threshold)
        
        coeffs_thresh.append((cH_thresh, cV_thresh, cD_thresh))
    
    # Reconstruct image
    denoised = pywt.waverec2(coeffs_thresh, wavelet)
    
    # Ensure same size as input (waverec2 can sometimes add padding)
    denoised = denoised[:noisy_image.shape[0], :noisy_image.shape[1]]
    
    # Clip to valid range
    denoised = np.clip(denoised, 0, 1)
    
    return denoised

### 4.1 Apply Denoising to All Noisy Images

In [ ]:
# Denoise with both hard and soft thresholding
wavelet_type = 'db4'  # Daubechies 4
decomp_level = 3

results = {}
for snr in snr_levels:
    results[snr] = {
        'noisy': noisy_images[snr],
        'hard': wavelet_denoise(noisy_images[snr], wavelet_type, decomp_level, 'hard'),
        'soft': wavelet_denoise(noisy_images[snr], wavelet_type, decomp_level, 'soft')
    }

# Display results for each SNR level
for snr in snr_levels:
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    
    # Original
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original')
    axes[0].axis('off')
    
    # Noisy
    noisy = results[snr]['noisy']
    psnr_noisy = peak_signal_noise_ratio(image, noisy)
    ssim_noisy = structural_similarity(image, noisy, data_range=1.0)
    axes[1].imshow(noisy, cmap='gray')
    axes[1].set_title(f'Noisy (SNR={snr}dB)\nPSNR={psnr_noisy:.2f}dB\nSSIM={ssim_noisy:.3f}')
    axes[1].axis('off')
    
    # Hard threshold
    hard = results[snr]['hard']
    psnr_hard = peak_signal_noise_ratio(image, hard)
    ssim_hard = structural_similarity(image, hard, data_range=1.0)
    axes[2].imshow(hard, cmap='gray')
    axes[2].set_title(f'Hard Threshold\nPSNR={psnr_hard:.2f}dB\nSSIM={ssim_hard:.3f}')
    axes[2].axis('off')
    
    # Soft threshold
    soft = results[snr]['soft']
    psnr_soft = peak_signal_noise_ratio(image, soft)
    ssim_soft = structural_similarity(image, soft, data_range=1.0)
    axes[3].imshow(soft, cmap='gray')
    axes[3].set_title(f'Soft Threshold\nPSNR={psnr_soft:.2f}dB\nSSIM={ssim_soft:.3f}')
    axes[3].axis('off')
    
    plt.suptitle(f'Denoising Results for SNR = {snr} dB (Wavelet: {wavelet_type}, Level: {decomp_level})', 
                 fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    print(f"\nSNR = {snr} dB:")
    print(f"  Hard Threshold - PSNR: {psnr_hard:.2f} dB, SSIM: {ssim_hard:.4f}")
    print(f"  Soft Threshold - PSNR: {psnr_soft:.2f} dB, SSIM: {ssim_soft:.4f}")
    print(f"  Improvement (Hard): {psnr_hard - psnr_noisy:.2f} dB")
    print(f"  Improvement (Soft): {psnr_soft - psnr_noisy:.2f} dB")

## 5. Parameter Study

### 5.1 Different Wavelet Types

In [ ]:
# Test different wavelet families
wavelets = ['haar', 'db4', 'db8', 'sym4', 'sym8', 'coif1', 'coif2']
test_snr = 10  # Use 10 dB noisy image for testing
test_image = noisy_images[test_snr]

print(f"Comparing different wavelets (SNR={test_snr} dB, Level=3, Soft thresholding):\n")
print(f"{'Wavelet':<10} {'PSNR (dB)':<12} {'SSIM':<10}")
print("-" * 35)

wavelet_results = {}
for wav in wavelets:
    try:
        denoised = wavelet_denoise(test_image, wav, 3, 'soft')
        psnr = peak_signal_noise_ratio(image, denoised)
        ssim = structural_similarity(image, denoised, data_range=1.0)
        wavelet_results[wav] = {'denoised': denoised, 'psnr': psnr, 'ssim': ssim}
        print(f"{wav:<10} {psnr:<12.2f} {ssim:<10.4f}")
    except Exception as e:
        print(f"{wav:<10} Error: {str(e)}")

# Visualize best performing wavelets
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

axes[0].imshow(test_image, cmap='gray')
psnr_noisy = peak_signal_noise_ratio(image, test_image)
axes[0].set_title(f'Noisy Image\nPSNR={psnr_noisy:.2f}dB')
axes[0].axis('off')

for i, wav in enumerate(wavelets):
    if wav in wavelet_results:
        axes[i+1].imshow(wavelet_results[wav]['denoised'], cmap='gray')
        axes[i+1].set_title(f'{wav}\nPSNR={wavelet_results[wav]["psnr"]:.2f}dB\nSSIM={wavelet_results[wav]["ssim"]:.3f}')
        axes[i+1].axis('off')

plt.suptitle(f'Wavelet Type Comparison (SNR={test_snr}dB, Soft Thresholding)', fontsize=14)
plt.tight_layout()
plt.show()

### 5.2 Different Decomposition Levels

In [ ]:
# Test different decomposition levels
levels = [1, 2, 3, 4, 5]
best_wavelet = 'db4'  # Use a good performing wavelet

print(f"Comparing decomposition levels (Wavelet={best_wavelet}, SNR={test_snr} dB, Soft thresholding):\n")
print(f"{'Level':<8} {'PSNR (dB)':<12} {'SSIM':<10}")
print("-" * 35)

level_results = {}
for lev in levels:
    denoised = wavelet_denoise(test_image, best_wavelet, lev, 'soft')
    psnr = peak_signal_noise_ratio(image, denoised)
    ssim = structural_similarity(image, denoised, data_range=1.0)
    level_results[lev] = {'denoised': denoised, 'psnr': psnr, 'ssim': ssim}
    print(f"{lev:<8} {psnr:<12.2f} {ssim:<10.4f}")

# Plot PSNR vs Level
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

psnrs = [level_results[lev]['psnr'] for lev in levels]
ssims = [level_results[lev]['ssim'] for lev in levels]

axes[0].plot(levels, psnrs, 'o-', linewidth=2, markersize=8)
axes[0].set_xlabel('Decomposition Level')
axes[0].set_ylabel('PSNR (dB)')
axes[0].set_title('PSNR vs Decomposition Level')
axes[0].grid(True, alpha=0.3)

axes[1].plot(levels, ssims, 'o-', linewidth=2, markersize=8, color='orange')
axes[1].set_xlabel('Decomposition Level')
axes[1].set_ylabel('SSIM')
axes[1].set_title('SSIM vs Decomposition Level')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 5.3 Hard vs Soft Thresholding Comparison

In [ ]:
# Compare hard vs soft for different noise levels
print(f"Hard vs Soft Thresholding (Wavelet={best_wavelet}, Level=3):\n")
print(f"{'SNR (dB)':<10} {'Method':<8} {'PSNR (dB)':<12} {'SSIM':<10}")
print("-" * 45)

for snr in snr_levels:
    hard = results[snr]['hard']
    soft = results[snr]['soft']
    
    psnr_hard = peak_signal_noise_ratio(image, hard)
    ssim_hard = structural_similarity(image, hard, data_range=1.0)
    psnr_soft = peak_signal_noise_ratio(image, soft)
    ssim_soft = structural_similarity(image, soft, data_range=1.0)
    
    print(f"{snr:<10} {'Hard':<8} {psnr_hard:<12.2f} {ssim_hard:<10.4f}")
    print(f"{'':<10} {'Soft':<8} {psnr_soft:<12.2f} {ssim_soft:<10.4f}")
    print()

## 6. Translation Invariant Wavelet Transform (TI-WT)

The standard wavelet transform is not translation invariant, which can lead to artifacts in denoised images. The Translation Invariant Wavelet Transform (also called cycle spinning or stationary wavelet transform) overcomes this by:

1. Shifting the image by different amounts
2. Denoising each shifted version
3. Shifting back and averaging the results

This reduces artifacts and improves denoising performance.

In [ ]:
def translation_invariant_denoise(noisy_image, wavelet='db4', level=3, 
                                  threshold_mode='soft', n_shifts=8):
    """
    Translation Invariant Wavelet Denoising using cycle spinning
    
    Parameters:
    -----------
    noisy_image : ndarray
        Noisy input image
    wavelet : str
        Wavelet type
    level : int
        Decomposition level
    threshold_mode : str
        'soft' or 'hard' thresholding
    n_shifts : int
        Number of shifts to use (along each axis)
    
    Returns:
    --------
    denoised : ndarray
        Denoised image
    """
    h, w = noisy_image.shape
    denoised_sum = np.zeros_like(noisy_image)
    count = 0
    
    # Try different circular shifts
    shifts = range(0, n_shifts)
    
    for shift_h in shifts:
        for shift_w in shifts:
            # Circular shift
            shifted = np.roll(noisy_image, (shift_h, shift_w), axis=(0, 1))
            
            # Denoise shifted image
            denoised_shifted = wavelet_denoise(shifted, wavelet, level, threshold_mode)
            
            # Shift back
            denoised_unshifted = np.roll(denoised_shifted, (-shift_h, -shift_w), axis=(0, 1))
            
            # Accumulate
            denoised_sum += denoised_unshifted
            count += 1
    
    # Average all shifted versions
    denoised = denoised_sum / count
    
    return denoised

def stationary_wavelet_denoise(noisy_image, wavelet='db4', level=3, threshold_mode='soft'):
    """
    Alternative implementation using Stationary Wavelet Transform (SWT)
    SWT is inherently translation invariant
    """
    # Estimate noise level
    sigma = estimate_noise_sigma(noisy_image, wavelet)
    
    # Calculate threshold
    N = noisy_image.size
    threshold = sigma * np.sqrt(2 * np.log(N))
    
    # Check if image size is compatible with SWT (must be divisible by 2^level)
    h, w = noisy_image.shape
    pad_h = (2**level - h % 2**level) % 2**level
    pad_w = (2**level - w % 2**level) % 2**level
    
    # Pad image if needed
    if pad_h > 0 or pad_w > 0:
        padded = np.pad(noisy_image, ((0, pad_h), (0, pad_w)), mode='reflect')
    else:
        padded = noisy_image
    
    # Perform stationary wavelet decomposition
    # swt2 returns a list of (cA, (cH, cV, cD)) tuples, one for each level
    coeffs = pywt.swt2(padded, wavelet, level=level)
    
    # Apply thresholding
    # Note: coeffs[i] = (cA, (cH, cV, cD)) for level i
    coeffs_thresh = []
    for coeff_level in coeffs:
        cA = coeff_level[0]
        cH, cV, cD = coeff_level[1]
        
        if threshold_mode == 'soft':
            cH_thresh = soft_threshold(cH, threshold)
            cV_thresh = soft_threshold(cV, threshold)
            cD_thresh = soft_threshold(cD, threshold)
        else:
            cH_thresh = hard_threshold(cH, threshold)
            cV_thresh = hard_threshold(cV, threshold)
            cD_thresh = hard_threshold(cD, threshold)
        
        coeffs_thresh.append((cA, (cH_thresh, cV_thresh, cD_thresh)))
    
    # Reconstruct
    denoised = pywt.iswt2(coeffs_thresh, wavelet)
    
    # Remove padding and ensure correct size
    denoised = denoised[:h, :w]
    denoised = np.clip(denoised, 0, 1)
    
    return denoised

### 6.1 Apply TI-WT Denoising

In [ ]:
# Apply both cycle spinning and SWT
print("Applying Translation Invariant Denoising...\n")

ti_results = {}
for snr in snr_levels:
    print(f"Processing SNR = {snr} dB...")
    
    # Cycle spinning (use fewer shifts for speed)
    ti_cycle = translation_invariant_denoise(noisy_images[snr], 'db4', 3, 'soft', n_shifts=4)
    
    # Stationary wavelet transform
    ti_swt = stationary_wavelet_denoise(noisy_images[snr], 'db4', 3, 'soft')
    
    ti_results[snr] = {
        'cycle': ti_cycle,
        'swt': ti_swt
    }

print("\nDone!")

### 6.2 Compare TI-WT with Standard Wavelet Denoising

In [ ]:
# Comprehensive comparison
for snr in snr_levels:
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Row 1: Original, Noisy, Standard DWT
    axes[0, 0].imshow(image, cmap='gray')
    axes[0, 0].set_title('Original')
    axes[0, 0].axis('off')
    
    noisy = noisy_images[snr]
    psnr_noisy = peak_signal_noise_ratio(image, noisy)
    ssim_noisy = structural_similarity(image, noisy, data_range=1.0)
    axes[0, 1].imshow(noisy, cmap='gray')
    axes[0, 1].set_title(f'Noisy (SNR={snr}dB)\nPSNR={psnr_noisy:.2f}dB, SSIM={ssim_noisy:.3f}')
    axes[0, 1].axis('off')
    
    dwt_soft = results[snr]['soft']
    psnr_dwt = peak_signal_noise_ratio(image, dwt_soft)
    ssim_dwt = structural_similarity(image, dwt_soft, data_range=1.0)
    axes[0, 2].imshow(dwt_soft, cmap='gray')
    axes[0, 2].set_title(f'Standard DWT (Soft)\nPSNR={psnr_dwt:.2f}dB, SSIM={ssim_dwt:.3f}')
    axes[0, 2].axis('off')
    
    # Row 2: Cycle spinning, SWT, Error comparison
    ti_cycle = ti_results[snr]['cycle']
    psnr_cycle = peak_signal_noise_ratio(image, ti_cycle)
    ssim_cycle = structural_similarity(image, ti_cycle, data_range=1.0)
    axes[1, 0].imshow(ti_cycle, cmap='gray')
    axes[1, 0].set_title(f'TI-WT (Cycle Spinning)\nPSNR={psnr_cycle:.2f}dB, SSIM={ssim_cycle:.3f}')
    axes[1, 0].axis('off')
    
    ti_swt = ti_results[snr]['swt']
    psnr_swt = peak_signal_noise_ratio(image, ti_swt)
    ssim_swt = structural_similarity(image, ti_swt, data_range=1.0)
    axes[1, 1].imshow(ti_swt, cmap='gray')
    axes[1, 1].set_title(f'TI-WT (SWT)\nPSNR={psnr_swt:.2f}dB, SSIM={ssim_swt:.3f}')
    axes[1, 1].axis('off')
    
    # Error map comparison
    error_dwt = np.abs(image - dwt_soft)
    error_swt = np.abs(image - ti_swt)
    error_diff = error_dwt - error_swt
    
    im = axes[1, 2].imshow(error_diff, cmap='RdBu_r', vmin=-0.2, vmax=0.2)
    axes[1, 2].set_title('Error Difference\n(DWT - SWT)\nBlue = SWT Better')
    axes[1, 2].axis('off')
    plt.colorbar(im, ax=axes[1, 2], fraction=0.046)
    
    plt.suptitle(f'Translation Invariant Wavelet Denoising Comparison (SNR={snr}dB)', 
                 fontsize=14, y=0.98)
    plt.tight_layout()
    plt.show()
    
    print(f"\n=== SNR = {snr} dB ===")
    print(f"Noisy:              PSNR={psnr_noisy:.2f}dB, SSIM={ssim_noisy:.4f}")
    print(f"Standard DWT:       PSNR={psnr_dwt:.2f}dB, SSIM={ssim_dwt:.4f} (Δ={psnr_dwt-psnr_noisy:+.2f}dB)")
    print(f"TI-WT (Cycle):      PSNR={psnr_cycle:.2f}dB, SSIM={ssim_cycle:.4f} (Δ={psnr_cycle-psnr_noisy:+.2f}dB)")
    print(f"TI-WT (SWT):        PSNR={psnr_swt:.2f}dB, SSIM={ssim_swt:.4f} (Δ={psnr_swt-psnr_noisy:+.2f}dB)")
    print(f"\nImprovement over DWT:")
    print(f"  Cycle Spinning:   {psnr_cycle - psnr_dwt:+.2f}dB")
    print(f"  SWT:              {psnr_swt - psnr_dwt:+.2f}dB")
    print()

## 7. Summary and Discussion

### Key Findings:

1. **Wavelet Type:**
   - Daubechies wavelets (db4, db8) typically perform well for natural images
   - Symlets (sym4, sym8) are also good choices
   - Haar wavelet is simple but may produce blocky artifacts
   - Higher order wavelets capture more detail but may be sensitive to noise

2. **Decomposition Levels:**
   - More levels capture coarser features but may over-smooth
   - Fewer levels preserve more detail but may not remove enough noise
   - Level 3-4 is typically a good compromise for most images

3. **Thresholding Type:**
   - **Soft thresholding:** Produces smoother results, better for high noise
   - **Hard thresholding:** Preserves edges better but may have artifacts
   - Soft thresholding generally gives higher PSNR/SSIM

4. **Translation Invariant Wavelets:**
   - Reduces artifacts from lack of shift-invariance in standard DWT
   - Typically improves PSNR by 0.5-2 dB
   - More computationally expensive (8-64x for cycle spinning)
   - SWT is faster than cycle spinning and often performs better
   - Particularly effective at preserving edges and textures

## 8. Final Comparison Table

In [ ]:
# Create comprehensive results table
print("="*80)
print("COMPREHENSIVE DENOISING RESULTS")
print("="*80)
print(f"\n{'SNR (dB)':<10} {'Method':<20} {'PSNR (dB)':<12} {'SSIM':<10} {'Improvement':<12}")
print("-"*80)

for snr in snr_levels:
    noisy = noisy_images[snr]
    psnr_noisy = peak_signal_noise_ratio(image, noisy)
    ssim_noisy = structural_similarity(image, noisy, data_range=1.0)
    
    methods = [
        ('Noisy', noisy),
        ('DWT Hard', results[snr]['hard']),
        ('DWT Soft', results[snr]['soft']),
        ('TI-WT Cycle', ti_results[snr]['cycle']),
        ('TI-WT SWT', ti_results[snr]['swt'])
    ]
    
    for method_name, img in methods:
        psnr = peak_signal_noise_ratio(image, img)
        ssim = structural_similarity(image, img, data_range=1.0)
        improvement = psnr - psnr_noisy
        
        print(f"{snr:<10} {method_name:<20} {psnr:<12.2f} {ssim:<10.4f} {improvement:+.2f} dB")
        
        if method_name == 'Noisy':
            snr = ''  # Only print SNR once per group
    
    print("-"*80)

print("\n" + "="*80)